# 百度网盘AI大赛-表格检测进阶：表格的结构化赛第10名方案

# 1. 数据集准备
观察数据集, 以及预测结果得出数据集存在一下问题
1. 非自然场景数据量偏少
2. 当非自然场景的数据存在, 模糊, 倾斜, 遮挡物时识别准确率偏低
根据这两个问题, 我们结合了 https://github.com/Belval/TextRecognitionDataGenerator 和 https://github.com/WenmuZhou/TableGeneration 这两个项目, 并开源到了 https://github.com/moyueheng/TableGeneration 进行数据生成, 获得了模糊, 倾斜的数据

链接: https://pan.baidu.com/s/1x0ZxC-2bz7Oy_iRrw0VwBw?pwd=grit 提取码: grit 

将数据集下载到 data 文件夹

In [2]:
import os
from os.path import join as pjoin
workr_space = os.getcwd()
data_dir = pjoin(workr_space, 'data')

In [4]:
# 解压
!unzip -oq data/testA.zip -d data
!unzip -oq data/train.zip -d data
!unzip -oq data/gen_table_2.5k.zip -d data

In [5]:
# 合并成一个扩展数据集

# 合并标注文件
gen_annos_path = pjoin(data_dir, 'gen_table_2.5k/gen_annos.txt') 
annos_path = pjoin(data_dir, 'train/annos.txt') 
ext_annos_path = pjoin(data_dir, 'ext_annos.txt') 

import json
with open(gen_annos_path, 'r') as f:
    gen_annos = json.load(f)

with open(annos_path, 'r') as f:
    annos = json.load(f)
annos.update(gen_annos)
with open(ext_annos_path, 'w') as f:
    json.dump(annos, f)

# 合并标注图片
from tqdm import tqdm
import os
ext_imgs_dir =  pjoin(data_dir, 'ext_imgs')
gen_imgs_dir =  pjoin(data_dir, 'gen_table_2.5k/img')
train_imgs_dir =  pjoin(data_dir, 'train/imgs')

os.makedirs(ext_imgs_dir, exist_ok=True)

import os
import shutil
def copy_images(src_dir, dst_dir):
    # 获取源目录下所有文件名
    file_list = os.listdir(src_dir)
    # 循环复制每个文件到目标目录
    for file_name in file_list:
        src_file = os.path.join(src_dir, file_name)
        dst_file = os.path.join(dst_dir, file_name)
        shutil.copy2(src_file, dst_file)


copy_images(gen_imgs_dir, ext_imgs_dir)
copy_images(train_imgs_dir,ext_imgs_dir)


# 2.下载训练代码

In [6]:
!git clone https://gitee.com/paddlepaddle/PaddleDetection.git

fatal: destination path 'PaddleDetection' already exists and is not an empty directory.


In [7]:
# 安装其他依赖
%cd ./PaddleDetection
! pip install -r requirements.txt
%cd ..

/jinx/table_struct/baiduwangpan_AI_tablestruct_10/PaddleDetection
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
/jinx/table_struct/baiduwangpan_AI_tablestruct_10


# 3.训练集转成voc

In [8]:
# 转voc
# thanks to https://blog.csdn.net/hu694028833/article/details/81089959

import xml.dom.minidom as minidom
import cv2
import json
from tqdm import tqdm
from os.path import join as pjoin


def anno_to_voc(annos_path, img_dir, annotation_dir, train_path):
    """
    Args:
        annos_path (str): 官方默认标注格式
        img_dir (str): 图片所在目录
        annotation_dir (str): voc格式保存, annotation文件夹路径
        train_path (str): voc格式保存, train文件路径
    """
    with open(annos_path, 'r') as f:
        data = json.load(f)
    os.makedirs(annotation_dir, exist_ok=True)
    def to_voc(train_path, annotation_dir, img_dir, data):
        with open(train_path, 'w') as fw:
            for name in tqdm(data.keys()):
                img_path = pjoin(img_dir, name)
                save_path = pjoin(annotation_dir, name.split('.')[0]+'.xml')
                try:
                    # generate xml
                    img = cv2.imread(img_path)
                    h,w,c = img.shape

                    dom = minidom.getDOMImplementation().createDocument(None,'annotations',None)
                    root = dom.documentElement

                    element = dom.createElement('filename')
                    element.appendChild(dom.createTextNode(img_path.split('/')[-1]))
                    root.appendChild(element)

                    element = dom.createElement('size')
                    element_c = dom.createElement('width')
                    element_c.appendChild(dom.createTextNode(str(w)))
                    element.appendChild(element_c)
                    element_c = dom.createElement('height')
                    element_c.appendChild(dom.createTextNode(str(h)))
                    element.appendChild(element_c)
                    element_c = dom.createElement('depth')
                    element_c.appendChild(dom.createTextNode(str(c)))
                    element.appendChild(element_c)
                    root.appendChild(element)

                    objects = data[name]
                    for a_object in objects:
                        element = dom.createElement('object')

                        element_c = dom.createElement('name')
                        element_c.appendChild(dom.createTextNode(a_object['label']))
                        element.appendChild(element_c)

                        element_c = dom.createElement('bndbox')
                        
                        element.appendChild(element_c)
                        element_cc = dom.createElement('xmin')
                        element_cc.appendChild(dom.createTextNode(str(a_object['box'][0])))
                        element_c.appendChild(element_cc)
                        element_cc = dom.createElement('ymin')
                        element_cc.appendChild(dom.createTextNode(str(a_object['box'][1])))
                        element_c.appendChild(element_cc)
                        element_cc = dom.createElement('xmax')
                        element_cc.appendChild(dom.createTextNode(str(a_object['box'][2])))
                        element_c.appendChild(element_cc)
                        element_cc = dom.createElement('ymax')
                        element_cc.appendChild(dom.createTextNode(str(a_object['box'][3])))

                        element_c.appendChild(element_cc)

                        root.appendChild(element)
                    with open(save_path, 'w', encoding='utf-8') as f:
                        dom.writexml(f, addindent='\t', newl='\n',encoding='utf-8')
                except:
                    print(name)
                    continue
                fw.write(img_path+' '+save_path+'\n')

    to_voc(train_path, annotation_dir, img_dir, data)

# 官方数据集转成voc
anno_to_voc(annos_path, train_imgs_dir, pjoin(data_dir, 'train/annotations'), pjoin(data_dir, 'train.txt'))
# 扩展数据集转成voc
anno_to_voc(ext_annos_path, ext_imgs_dir, pjoin(data_dir, 'ext_annotations'), pjoin(data_dir, 'ext_train.txt'))

# 写入lable.txt文件
with open(pjoin(data_dir, 'label_list.txt'), 'w') as f:
    f.write("table\nrow\ncolumn\nspanning_cell")

100%|██████████| 10241/10241 [01:16<00:00, 134.38it/s]


# 4.训练
训练模型我们使用的是ppyoloe_plus
先使用预训练模型 https://bj.bcebos.com/v1/paddledet/models/pretrained/ppyoloe_crn_l_obj365_pretrained.pdparams 和 官方数据集训练80e, 再使用扩展数据集训练100e

注意:
    记得修改配置文件的一些路径
    ![数据集目录修改](./img/%E6%95%B0%E6%8D%AE%E9%9B%86%E7%9B%AE%E5%BD%95%E4%BF%AE%E6%94%B9.png)

In [9]:
# 复制训练配置文件
myPPYoloeVoce_60e_path = pjoin(workr_space, 'configs/myPPYoloeVoce_60e.yml')
ext_myPPYoloeVoce_120e_path = pjoin(workr_space, 'configs/ext_myPPYoloeVoce_120e.yml')
config_dir = pjoin(workr_space, 'PaddleDetection/configs/ppyoloe/')
shutil.copy2(myPPYoloeVoce_60e_path, config_dir)
shutil.copy2(ext_myPPYoloeVoce_120e_path, config_dir)

# 复制数据集配置文件
myPPYoloeVoce_60e_voc_path = pjoin(workr_space, 'configs/voc.yml')
myPPYoloeVoce_60e_ext_voc_path = pjoin(workr_space, 'configs/ext_voc.yml')
dataset_dir = pjoin(workr_space, 'PaddleDetection/configs/datasets/')
shutil.copy2(myPPYoloeVoce_60e_voc_path, dataset_dir)
shutil.copy2(myPPYoloeVoce_60e_ext_voc_path, dataset_dir)

'/jinx/table_struct/baiduwangpan_AI_tablestruct_10/PaddleDetection/configs/datasets/ext_voc.yml'

In [10]:
# 训练官方数据集60e
%cd PaddleDetection
! export CUDA_VISIBLE_DEVICES=0 #windows和Mac下不需要执行该命令
! python tools/train.py -c configs/ppyoloe/myPPYoloeVoce_60e.yml
%cd ..

/jinx/table_struct/baiduwangpan_AI_tablestruct_10/PaddleDetection
grep: warning: GREP_OPTIONS is deprecated; please use an alias or script
/root/.local/lib/python3.7/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/root/.local/lib/python3.7/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/root/.local/lib/python3.7/site-packages/pkg_resources/__init__.py:28

In [ ]:
# 训练扩展数据集120e
%cd PaddleDetection
! export CUDA_VISIBLE_DEVICES=0 #windows和Mac下不需要执行该命令
! python tools/train.py -c configs/ppyoloe/ext_myPPYoloeVoce_120e.yml
%cd ..

In [3]:
# 模型导出
%cd PaddleDetection
# 注意修改output_dir目录的路径, 这是你模型输出的目录
! python tools/export_model.py -c configs/ppyoloe/ext_myPPYoloeVoce_120e.yml  --output_dir=baiduwangpan_AI_tablestruct_10/mymodel/  -o weights=output/ext_myPPYoloeVoce_120e/model_final
%cd ..

/jinx/table_struct/PaddleDetection
grep: warning: GREP_OPTIONS is deprecated; please use an alias or script
/root/.local/lib/python3.7/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/root/.local/lib/python3.7/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/root/.local/lib/python3.7/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Depreca

# 一个预测脚本

In [3]:

import paddle
import cv2
import numpy as np
import os
from os.path import join as pjoin

# model = paddle.jit.load('/jinx/table_struct/mymodel/adjust_ext_myPPYoloeVoc_40e/model')
model = paddle.jit.load(pjoin(workr_space, 'mymodel/ext_myPPYoloeVoce_120e/model'))
                        
model.eval()

test_img_dir = pjoin(data_dir, 'pubtest/imgs') 

pre_li = list()


def pre_img(test_img_path):
    img = cv2.imread(test_img_path)
    h, w, c = img.shape

    input_size = (640,640)
    scale_factor = [input_size[0] / img.shape[0], input_size[1] / img.shape[1]]
    factor = np.array(scale_factor, dtype=np.float32)
    factor = paddle.to_tensor(factor).reshape((1, 2)).astype('float32')

    img = cv2.resize(img, (640, 640))
    img = img/255
    # 第一个列表是mean, 第二个列表是std
    img = (img-np.array([0.0, 0.0, 0.0]))/np.array([1.0, 1.0, 1.0])
    img = img.transpose([2,0,1])
    img = paddle.to_tensor(img).astype('float32')
    img = paddle.reshape(img,[1]+img.shape)
    pre = model(img, factor)
    return pre

test_img_path_li = list()
test_img_name_li = list()

test_img_name_li = os.listdir(test_img_dir)
test_img_path_li = [ os.path.join(test_img_dir, test_img) for test_img in test_img_name_li]
pre_li = list()

for test_img_path in test_img_path_li:
    pre = pre_img(test_img_path)
    pre_li.append(pre)



W0304 14:22:25.126752 16308 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0304 14:22:25.129945 16308 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


In [8]:
# 测试效果可视化
"""
table
row
column
spanning_cell
"""

color_dict = {
    0.0 : (0, 0, 0), # table 黑色
    1.0 :  (169, 169, 169), # row grey
    2.0 : (216, 99, 67), #  column bulue 
    3.0 :   (25, 225, 255), # spanning_cell 黄色
}
# 
test_A_result_dir =  pjoin(workr_space, 'ext_myPPYoloeVoc_120e_result')

if not os.path.exists(test_A_result_dir):
    os.makedirs(test_A_result_dir)

for i in range(len(pre_li)):

    pre= pre_li[i]
    img = cv2.imread(test_img_path_li[i])
    
    for item in pre[0].numpy():
        lable_index, value, xmin, ymin, xmax, ymax = item
        xmin ,ymin, xmax, ymax = [int(x) for x in [xmin ,ymin, xmax, ymax]]
        # if value > 0.46:
        if value > 0.52: # ext_myPPYoloeVoc_l_100e_test_A_result
            thickness = 3
            if lable_index == 0.0:
                thickness = 20
            img = cv2.rectangle(img, (xmin, ymin), (xmax,ymax),color = color_dict[lable_index], thickness=thickness)
    cv2.imwrite(os.path.join(test_A_result_dir, test_img_name_li[i]),img)


In [5]:
! python predict.py data/pubtest/imgs test_A_result

Error: Can not import paddle core while this file exists: /usr/local/lib/python3.7/dist-packages/paddle/fluid/libpaddle.so
grep: warning: GREP_OPTIONS is deprecated; please use an alias or script
Traceback (most recent call last):
  File "predict.py", line 9, in <module>
    import paddle
  File "/usr/local/lib/python3.7/dist-packages/paddle/__init__.py", line 25, in <module>
    from .framework import monkey_patch_variable
  File "/usr/local/lib/python3.7/dist-packages/paddle/framework/__init__.py", line 17, in <module>
    from . import random  # noqa: F401
  File "/usr/local/lib/python3.7/dist-packages/paddle/framework/random.py", line 16, in <module>
    import paddle.fluid as fluid
  File "/usr/local/lib/python3.7/dist-packages/paddle/fluid/__init__.py", line 36, in <module>
    from . import framework
  File "/usr/local/lib/python3.7/dist-packages/paddle/fluid/framework.py", line 37, in <module>
    from . import core
  File "/usr/local/lib/python3.7/dist-packages/paddle/fluid/co

In [6]:
# 打包提交
! zip -r ext_myPPYoloeVoc_120e_submit.zip predict.py mymodel/ext_myPPYoloeVoce_120e

  adding: predict.py (deflated 58%)
  adding: mymodel/ext_myPPYoloeVoce_120e/ (stored 0%)
  adding: mymodel/ext_myPPYoloeVoce_120e/model.pdmodel (deflated 93%)
  adding: mymodel/ext_myPPYoloeVoce_120e/model.pdiparams.info (deflated 83%)
  adding: mymodel/ext_myPPYoloeVoce_120e/infer_cfg.yml (deflated 37%)
  adding: mymodel/ext_myPPYoloeVoce_120e/model.pdiparams (deflated 7%)
